In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame

In [2]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [3]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [4]:
val sameModel = PipelineModel.load(modelPath)

sameModel = pipeline_678b938ed79d


pipeline_678b938ed79d

In [5]:
inputDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    //batchDF.show()
    val BatchPredictionsDF = sameModel.transform(batchDF)
        
    val ResOut = BatchPredictionsDF
        .withColumn("scoring_result", when($"prediction" === 1.0, "Negative").otherwise("Positive"))
        .select($"tweet",getProbability($"probability").alias("clean_probability"),$"scoring_result")
    
    ResOut.show()
    
}.start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@a045a61

+--------------------+-------------------+--------------+
|               tweet|  clean_probability|scoring_result|
+--------------------+-------------------+--------------+
|Why do my Sprint ...|0.48813715240666994|      Positive|
|twittter #F1 #fer...| 0.4893803141871178|      Positive|
|@edkaye Imagine b...| 0.5196840136613531|      Negative|
|@sliprywenfrosty ...| 0.4969768405936825|      Positive|
|Stupid England  s...| 0.5427444326775565|      Negative|
|realized one of m...|0.45466176299513994|      Positive|
|Poor show to the ...| 0.4726675977405969|      Positive|
|Sally is ill anyo...| 0.5036197978919443|      Negative|
|@iamjonathancook ...| 0.4709991139690263|      Positive|
|@TheRealCoryBach ...|0.48252267860945763|      Positive|
|@MacGuffen I'll b...| 0.5231576995756609|      Negative|
|@vaxen_var no, ju...| 0.5495938586111289|      Negative|
|@LadyParadis Good...| 0.5473370094195409|      Negative|
|@caitrinrose Haha...| 0.5477814421583571|      Negative|
|@MatthewWardl

+--------------------+-------------------+--------------+
|               tweet|  clean_probability|scoring_result|
+--------------------+-------------------+--------------+
|@thevideodog Went...|0.49198059190740595|      Positive|
|hates her PE clas...| 0.5058635844347673|      Negative|
|Searching #telstr...| 0.4925905340764459|      Positive|
|@Torae I passed o...| 0.4732515202060806|      Positive|
|@MrsWrustare good...| 0.4803308013095206|      Positive|
|@YESandME @Twiter...| 0.5142853652389118|      Negative|
|@firqin That's th...| 0.5460000084035831|      Negative|
|@CaraNinaMcfly I ...| 0.5785348345806881|      Negative|
|@swined Я пока не...| 0.5190682117293128|      Negative|
|sittin at work......| 0.5140241009572759|      Negative|
|@frogcooke that w...| 0.4869215003474326|      Positive|
|Gettin ready 2 go...| 0.5189303259819534|      Negative|
|good morning ever...| 0.5489351509447654|      Negative|
|@MsXXFAST sounds ...| 0.5206118917175135|      Negative|
+-------------

+--------------------+-------------------+--------------+
|               tweet|  clean_probability|scoring_result|
+--------------------+-------------------+--------------+
|@_Loch_Nessie_ my...| 0.5092488824176581|      Negative|
|sooo.. where is e...| 0.5190682117293128|      Negative|
|@lovesong09 noooo...| 0.4850082551049901|      Positive|
|@dorizinn No, peo...| 0.5128127696035197|      Negative|
|Don't no y I'm up...|0.44870935945295803|      Positive|
|Death is the most...|0.45950764075747036|      Positive|
|Ok this won't be ...|  0.518438976551519|      Negative|
|@abbietoeknee Goo...| 0.5292938134089862|      Negative|
|Got ready an hour...| 0.4901959220171824|      Positive|
+--------------------+-------------------+--------------+

+--------------------+-------------------+--------------+
|               tweet|  clean_probability|scoring_result|
+--------------------+-------------------+--------------+
|Monday morning......| 0.5152351426976857|      Negative|
|I already mi

In [6]:
//Вывод статисктики скользящим окном по 10 сек
val BatchPredictionsDF = sameModel.transform(inputDF)

val ResBatchPredictionsDF = BatchPredictionsDF
                  .withColumn("positive", when($"prediction" === 1.0, 1.0).otherwise(0))
                  .withColumn("negative", when($"prediction" === 0.0, 1.0).otherwise(0))
                  .withColumn("timestamp", current_timestamp())

val WindowedSum = ResBatchPredictionsDF.select("timestamp","positive","negative").groupBy(
              window($"timestamp", "10 seconds")
            ).sum().orderBy("window")

val ResOut = WindowedSum.writeStream
              .outputMode("complete")
              .format("console")
              .option("truncate", "false")
              .start()

BatchPredictionsDF = [tweet: string, words: array<string> ... 4 more fields]
ResBatchPredictionsDF = [tweet: string, words: array<string> ... 7 more fields]
WindowedSum = [window: struct<start: timestamp, end: timestamp>, sum(positive): double ... 1 more field]
ResOut = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@d460123


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@d460123

-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+-------------+-------------+
|window                                    |sum(positive)|sum(negative)|
+------------------------------------------+-------------+-------------+
|[2020-02-05 08:26:30, 2020-02-05 08:26:40]|10.0         |10.0         |
+------------------------------------------+-------------+-------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+-------------+-------------+
|window                                    |sum(positive)|sum(negative)|
+------------------------------------------+-------------+-------------+
|[2020-02-05 08:26:30, 2020-02-05 08:26:40]|10.0         |10.0         |
|[2020-02-05 08:27:20, 2020-02-05 08:27:30]|7.0          |3.0          |
+------------------------------------------+-------------+-------------+

-